In [1]:
# Imports
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

# Regression Analysis: Seasonal Effects with Sklearn Linear Regression

In [2]:
# Futures contract on the Yen-dollar exchange rate:
# This is the continuous chain of the futures contracts that are 1 month to expiration
yen_futures = pd.read_csv(
    Path("yen.csv"), index_col="Date", infer_datetime_format=True, parse_dates=True
)
yen_futures.head()

Open    High     Low    Last  Change  Settle  Volume  \
Date                                                                 
1976-08-02  3398.0  3401.0  3398.0  3401.0     NaN  3401.0     2.0   
1976-08-03  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-04  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-05  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   
1976-08-06  3401.0  3401.0  3401.0  3401.0     NaN  3401.0     0.0   

            Previous Day Open Interest  
Date                                    
1976-08-02                         1.0  
1976-08-03                         1.0  
1976-08-04                         1.0  
1976-08-05                         1.0  
1976-08-06                         1.0

In [12]:
# Trim the dataset to begin on January 1st, 1990
yen_futures_trim = yen_futures.loc["1990-01-01":, :]
yen_futures_trim.head()

Open    High     Low    Last  Change  Settle   Volume  \
Date                                                                  
1990-01-02  6954.0  6954.0  6835.0  6847.0     NaN  6847.0  48336.0   
1990-01-03  6877.0  6910.0  6865.0  6887.0     NaN  6887.0  38206.0   
1990-01-04  6937.0  7030.0  6924.0  7008.0     NaN  7008.0  49649.0   
1990-01-05  6952.0  6985.0  6942.0  6950.0     NaN  6950.0  29944.0   
1990-01-08  6936.0  6972.0  6936.0  6959.0     NaN  6959.0  19763.0   

            Previous Day Open Interest  
Date                                    
1990-01-02                     51473.0  
1990-01-03                     53860.0  
1990-01-04                     55699.0  
1990-01-05                     53111.0  
1990-01-08                     52072.0

# Data Preparation

### Returns and Lagged Returns

In [52]:
 # Create a series using "Settle" price percentage returns, drop any nan"s, and create a lagged return using the shift function:
yen_futures_trim["Return"] = yen_futures_trim.Settle.pct_change() * 100
yen_futures_trim['Lagged_Return'] = yen_futures_trim['Return'].shift()
yen_futures_trim = yen_futures_trim.dropna()
yen_futures_trim.tail() 

Open    High     Low    Last  Change  Settle    Volume  \
Date                                                                   
2019-10-09  9381.0  9391.5  9330.5  9343.5    38.5  9338.0   99153.0   
2019-10-10  9343.5  9380.5  9293.5  9301.0    34.5  9303.5  159397.0   
2019-10-11  9308.5  9309.0  9240.0  9267.0    52.5  9251.0  158810.0   
2019-10-14  9259.0  9292.0  9250.5  9261.0    14.0  9265.0   69457.0   
2019-10-15  9264.5  9280.0  9216.5  9220.0    43.5  9221.5  108342.0   

            Previous Day Open Interest  Diff    Return  Lagged_Return  
Date                                                                   
2019-10-09                    145470.0 -38.5 -0.410601       0.170931  
2019-10-10                    144474.0 -34.5 -0.369458      -0.410601  
2019-10-11                    147471.0 -52.5 -0.564304      -0.369458  
2019-10-14                    153902.0  14.0  0.151335      -0.564304  
2019-10-15                    151564.0 -43.5 -0.469509       0.151335

### Train Test Split

In [53]:
# Create a train/test split for the data using 2018-2019 for testing and the rest for training
train = yen_futures_trim[:'2017']
test = yen_futures_trim['2018':]

In [54]:
# Create four DataFrames:
# X_train (training set using just the independent variables), X_test (test set of of just the independent variables)
# Y_train (training set using just the "y" variable, i.e., "Futures Return"), Y_test (test set of just the "y" variable):
X_train = train["Lagged_Return"].to_frame()
y_train = train["Return"]
X_test = test["Lagged_Return"].to_frame()
y_test = test["Return"]

In [55]:
X_train

Lagged_Return
Date                     
2014-02-24      -0.204604
2014-02-25       0.020502
2014-02-26       0.327970
2014-02-27      -0.296251
2014-02-28       0.389344
...                   ...
2017-12-22       0.084621
2017-12-26       0.118370
2017-12-27       0.118230
2017-12-28      -0.151830
2017-12-29       0.180221

[963 rows x 1 columns]

# Linear Regression Model

In [56]:
# Create a Linear Regression model and fit it to the training data
from sklearn.linear_model import LinearRegression

# Create new dataframes:
X_train = train["Lagged_Return"].to_frame()
y_train = train["Return"]
X_test = test["Lagged_Return"].to_frame()
y_test = test["Return"]

# Fit a SKLearn linear regression using just the training set (X_train, Y_train):
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

# Make predictions using the Testing Data

Note: We want to evaluate the model using data that it has never seen before, in this case: X_test.

In [57]:
# Make a prediction of "y" values using just the test dataset
predictions = model.predict(X_test)

In [58]:
# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
Results = y_test.to_frame()
Results["Predicted Return"] = predictions
Results.head(2)

Return  Predicted Return
Date                                  
2018-01-02  0.297285         -0.008431
2018-01-03 -0.240479         -0.008696

In [59]:
# Plot the first 20 predictions vs the true values
Results[:20].plot(subplots=True)

array([<AxesSubplot:xlabel='Date'>, <AxesSubplot:xlabel='Date'>],
      dtype=object)

# Out-of-Sample Performance

Evaluate the model using "out-of-sample" data (X_test and y_test)

In [60]:
from sklearn.metrics import mean_squared_error, r2_score
# Calculate the mean_squared_error (MSE) on actual versus predicted test "y" 
mse = mean_squared_error(
    Results["Return"],
    Results["Predicted Return"]
)

# Using that mean-squared-error, calculate the root-mean-squared error (RMSE):
rmse = np.sqrt(mse)
print(f"Out-of-Sample Root Mean Squared Error (RMSE): {rmse}")

Out-of-Sample Root Mean Squared Error (RMSE): 0.41547588962821286


# In-Sample Performance

Evaluate the model using in-sample data (X_train and y_train)

In [61]:
# Construct a dataframe using just the "y" training data:
in_sample_results = y_train.to_frame()

# Add a column of "in-sample" predictions to that dataframe:  
in_sample_results["In-sample Predictions"] = model.predict(X_train)

# Calculate in-sample mean_squared_error (for comparison to out-of-sample)
in_sample_mse = mean_squared_error(
    in_sample_results["Return"],
    in_sample_results["In-sample Predictions"]
)

# Calculate in-sample root mean_squared_error (for comparison to out-of-sample)
in_sample_rmse = np.sqrt(in_sample_mse)
print(f"In-sample Root Mean Squared Error (RMSE): {in_sample_rmse}")

In-sample Root Mean Squared Error (RMSE): 0.5972597350419817


# Conclusions

Results

The RMSE is a measure of model accuracy. As such, when you have a higher RMSE, it means the accuracy is lower (since a higher RMSE means higher prediction error).

Our results show that the in-sample (training sample) RMSE, at 0.597, is higher than the out-of-sample (testing sample) at 0.415. Out-of-sample errors are typically higher than in-sample errors.

Analysis of Results:

A good model is reflected in similar RMSEs between the training and test sets.  Our training and test RMSEs are somewhat similar but not extremely close in range, so we can say that our model has fair predictive value.